In [2]:
!pip install transformers datasets peft trl torch evaluate accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127

In [5]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from trl import setup_chat_format

2025-08-19 07:03:59.620210: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755587039.796819      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755587039.853057      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [7]:
device=("cuda")
torch.cuda.is_available()

True

In [11]:
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [12]:
model, tokenizer= setup_chat_format(model=model, tokenizer=tokenizer)

In [10]:
message = [
    {"role": "user", "content": "Hello, How are you?" },
    {"role": "assistant", "content": "I'm doing well, thank you! How can I assist you today?"},
]

In [15]:
input_text = tokenizer.apply_chat_template(message, tokenize=False)
print(input_text)

<|im_start|>user
Hello, How are you?<|im_end|>
<|im_start|>assistant
I'm doing well, thank you! How can I assist you today?<|im_end|>



In [16]:
input_text = tokenizer.apply_chat_template(message, tokenize=True, add_generation_prompt=True)
print(input_text)

[1, 4093, 198, 19556, 28, 1073, 359, 346, 47, 2, 198, 1, 520, 9531, 198, 57, 5248, 2567, 876, 28, 9984, 346, 17, 1073, 416, 339, 4237, 346, 1834, 47, 2, 198, 1, 520, 9531, 198]


In [17]:
print("Conversation Decode :", tokenizer.decode(token_ids=input_text))

Conversation Decode : <|im_start|>user
Hello, How are you?<|im_end|>
<|im_start|>assistant
I'm doing well, thank you! How can I assist you today?<|im_end|>
<|im_start|>assistant



In [22]:
from IPython.core.display import display, HTML

display(
    HTML("""
<iframe
  src="https://huggingface.co/datasets/HuggingFaceTB/smoltalk/embed/viewer/all/train"
  frameborder="0"
  width="100%"
  height="560px"
></iframe>"""
        )
)

In [26]:
dataset = load_dataset("HuggingFaceTB/smoltalk", "everyday-conversations")

data/everyday-conversations/train-00000-(…):   0%|          | 0.00/946k [00:00<?, ?B/s]

data/everyday-conversations/test-00000-o(…):   0%|          | 0.00/52.6k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2260 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/119 [00:00<?, ? examples/s]

In [29]:
import pandas as pd
df = pd.DataFrame(dataset["train"])
df

,full_topic,messages
0,Travel/Vacation destinations/Beach resorts,"[{'content': 'Hi there', 'role': 'user'}, {'co..."
1,Work/Career development/Mentorship,"[{'content': 'Hi', 'role': 'user'}, {'content'..."
2,Shopping/Window shopping/Window shopping etiqu...,"[{'content': 'Hi', 'role': 'user'}, {'content'..."
3,Cooking/Cooking for others/Food gifting,"[{'content': 'Hi there', 'role': 'user'}, {'co..."
4,Weather/Climate change/Climate change impacts,"[{'content': 'Hi', 'role': 'user'}, {'content'..."
...,...,...
2255,Cooking/Cooking shows/Kids' cooking shows,"[{'content': 'Hi', 'role': 'user'}, {'content'..."
2256,Music/Musical instruments/Learning an instrument,"[{'content': 'Hi', 'role': 'user'}, {'content'..."
2257,basic math concepts/addition and subtraction,"[{'content': 'Hi', 'role': 'user'}, {'content'..."
2258,Pets/Veterinary visits/Specialized treatments,"[{'content': 'Hi there', 'role': 'user'}, {'co..."


In [53]:
def token_fun(row):
    text = tokenizer.apply_chat_template(
        row["messages"],
        tokenize=True,
        add_generation_prompt=True
    )
    row["tokens"] = text
    return row

tokenized_dataset = dataset.map(token_fun)

In [54]:
print(tokenized_dataset["train"][0])

{'full_topic': 'Travel/Vacation destinations/Beach resorts', 'messages': [{'content': 'Hi there', 'role': 'user'}, {'content': 'Hello! How can I help you today?', 'role': 'assistant'}, {'content': "I'm looking for a beach resort for my next vacation. Can you recommend some popular ones?", 'role': 'user'}, {'content': "Some popular beach resorts include Maui in Hawaii, the Maldives, and the Bahamas. They're known for their beautiful beaches and crystal-clear waters.", 'role': 'assistant'}, {'content': 'That sounds great. Are there any resorts in the Caribbean that are good for families?', 'role': 'user'}, {'content': 'Yes, the Turks and Caicos Islands and Barbados are excellent choices for family-friendly resorts in the Caribbean. They offer a range of activities and amenities suitable for all ages.', 'role': 'assistant'}, {'content': "Okay, I'll look into those. Thanks for the recommendations!", 'role': 'user'}, {'content': "You're welcome. I hope you find the perfect resort for your v

In [55]:
dtt = load_dataset("openai/gsm8k", "main")

README.md: 0.00B [00:00, ?B/s]

main/train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

main/test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [58]:
print(dtt)

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 7473
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 1319
    })
})


In [61]:
def wrap_fun(sample):
    message = [
        {"role": "user", "content": sample["question"]},
        {"role": "assistant", "content": sample["answer"]}
    ]
    chat_text = tokenizer.apply_chat_template(
        message,
        tokenize=False,
        add_generation_prompt=False
    )
    sample["message"] = message
    sample["chat_text"] = chat_text
    return sample

tk_dataset = dtt.map(wrap_fun)

Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

Map:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [63]:
print(tk_dataset["train"][1])

{'question': 'Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?', 'answer': 'Weng earns 12/60 = $<<12/60=0.2>>0.2 per minute.\nWorking 50 minutes, she earned 0.2 x 50 = $<<0.2*50=10>>10.\n#### 10', 'message': [{'content': 'Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?', 'role': 'user'}, {'content': 'Weng earns 12/60 = $<<12/60=0.2>>0.2 per minute.\nWorking 50 minutes, she earned 0.2 x 50 = $<<0.2*50=10>>10.\n#### 10', 'role': 'assistant'}], 'chat_text': '<|im_start|>user\nWeng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?<|im_end|>\n<|im_start|>assistant\nWeng earns 12/60 = $<<12/60=0.2>>0.2 per minute.\nWorking 50 minutes, she earned 0.2 x 50 = $<<0.2*50=10>>10.\n#### 10<|im_end|>\n'}
